In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
import dtreeviz
import graphviz
import graphviz.backend as be
from IPython.display import Image, display_svg, SVG
import warnings

warnings.filterwarnings("ignore", module="matplotlib\..*")

## BUSINESS QUESTIONS


*The major concern is the role of seasons in ISI rating. 
The common questions include, what season do most fires take place?
As fire prevention also might require the presence of active workforce for early detection, the absence of a workforce due to weekends was also evaluated. As we don´t have full control of the data details, it was hard to factor in public holidays and the normal work schedule of forest managers



In [ ]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import matplotlib.font_manager
fm = matplotlib.font_manager
fm._get_fontconfig_fonts.cache_clear()
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
df = pd.read_csv("forestfireexp.csv")
df

In [ ]:
#list of columns
df.columns

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df2=df.copy()

In [ ]:
#drop month and day
df=df.drop(['month', 'day'], axis=1)

In [ ]:
df.columns

## making sense of the data

The data focus is on the ISI which correlates to the velocity of fire spread. It´s related to the Forest Fire Weather Index which was a system developed in canada for rating the threat of forest fires , and has subsequently been proven to fit in the Mediterrenian forests (Cortez and Morias "A Data Mining Approach to Predict Forest Fires using Meteorological Data")

In [ ]:
# Question 1
# What role does season play in causing fires in public lands
#using ISI as a measure of fire danger rating, we can see the role of season in causing fires in public lands
#title of graph is "Fire Danger Rating by Season"

df['season'].value_counts().plot(kind='pie')
plt.title('Fire Recording by Season')



In [ ]:
#check ISI aavergae by season , assign different color shades of blue to each season
df.groupby('season')['ISI'].mean().plot(kind='bar', color=['orange', 'red', 'brown', 'teal'])
plt.title('ISI by Season')
plt.ylabel('ISI Rating')
plt.xlabel('Season')




In [ ]:
# check if ISI avergae per season is significantly different
from scipy import stats
spring = df[df['season']=='spring']['ISI']
summer = df[df['season']=='summer']['ISI']
autumn = df[df['season']=='autumn']['ISI']
winter = df[df['season']=='winter']['ISI']
stats.f_oneway(spring, summer, autumn, winter)

f_statistic, p_value = stats.f_oneway(spring, summer, autumn, winter)

print("F-statistic:", f_statistic)
print("p-value:", p_value)

alpha = 0.05
if p_value < alpha:
    print("The differences in ISI averages per season are statistically significant, so we accept that seasons play a significant role in forest fire.")
else:
    print(
        "The differences in ISI averages per season are not statistically significant."
    )

In [ ]:
df.columns

In [ ]:
# check week day and weekend 

df.groupby("day_category")["ISI"].mean().plot(kind="bar")
plt.title("ISI Average by weekend or weekday")

In [ ]:
# check if ISI avergae per day_category is significantly different
weekday = df[df['day_category']=='weekday']['ISI']
weekend = df[df['day_category']=='weekend']['ISI']
stats.f_oneway(weekday, weekend)

f_statistic, p_value = stats.f_oneway(weekday, weekend)

print("F-statistic:", f_statistic)
print("p-value:", p_value)

alpha = 0.05
if p_value < alpha:
    print(
        "The differences in ISI averages per weekday or weekend are statistically significant, so we accept that the day of the week play a significant role in forest fire."
    )
else:
    print(
        "The differences in ISI averages per weekday or weekend are not statistically significant."
    )

In [ ]:
# check area average by day_category
colors = ["blue", "orange"]
df.groupby("day_category")["area"].mean().plot(kind="bar", color=colors)
plt.title("Area Average by weekend or weekday")
plt.ylabel("Area Average")  
plt.xlabel("Day Category")    
plt.show()

we can see the fire covers more area during weekends , which means we would need to consider rapid emergency response for weekends

In [ ]:
# let's check if the area average per day_category is significantly different
weekday = df[df['day_category']=='weekday']['area']
weekend = df[df['day_category']=='weekend']['area']
stats.f_oneway(weekday, weekend)

f_statistic, p_value = stats.f_oneway(weekday, weekend)

print("F-statistic:", f_statistic)
print("p-value:", p_value)

alpha = 0.05
if p_value < alpha:
    print(
        "The differences in burnt areas averages per weekday or weekend are statistically significant, so we accept that workdays play a significant role in forest fire."
    )
else:
    print(
        "The differences in burnt areas averages per weekday or weekend are not statistically significant."
    )

## DATA MODELLING

now to data modelling to predict using decision tree

First we would try to predict the ISI under all conditions , secondly we will attempt to predict total affected area 

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
import dtreeviz
import graphviz
import graphviz.backend as be
from IPython.display import Image, display_svg, SVG
import warnings

warnings.filterwarnings("ignore", module="matplotlib\..*")

In [ ]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import matplotlib.font_manager
fm = matplotlib.font_manager
fm._get_fontconfig_fonts.cache_clear()
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
#view the data
df.head()

In [ ]:
df.shape

In [ ]:
x=df.drop(columns=['ISI'])
y=df['ISI']

In [ ]:
x.columns

In [ ]:
x2=x.copy()

In [ ]:
#assing 1 to weekend and 0 to weekday, 1 to summer , 2 to autumn, 3 to winter and 4 to spring
x2['day_category']=x2['day_category'].map({'weekday':0, 'weekend':1})
x2['season']=x2['season'].map({'summer':1, 'autumn':2, 'winter':3, 'spring':4})

x2.head()

In [ ]:
#sunique value in season
x2['season'].unique()

In [ ]:
#check for skewness
x2.skew()


we see skewness in FFMC , DC, rain , area and season. 
we will transform those coliumns to correct skewness

In [ ]:
#target ffmc, dc, rain , are and season are skewed, we will use power transformer to transform the data
from sklearn.preprocessing import PowerTransformer
pt=PowerTransformer()
x2[['FFMC', 'DC', 'rain', 'area']]=pt.fit_transform(x2[['FFMC', 'DC', 'rain', 'area']])
x2.skew()


In [ ]:
import numpy as np

In [ ]:
#check skewness
x2.skew()

In [ ]:
#check multicollinearity
import seaborn as sns
plt.figure(figsize=(10,10))
sns.heatmap(x2.corr(), annot=True)


In [ ]:
x.describe().T

In [ ]:
#minmax scaling
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X=scaler.fit_transform(x2)



In [ ]:
#convert to dataframe
X=pd.DataFrame(X, columns=x2.columns)
X.describe().T

In [ ]:
#split X and y into training and testing sets
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

#view the shape of the training and testing sets
X_train_df = pd.DataFrame(X_train, columns=x2.columns)
X_test_df = pd.DataFrame(X_test, columns=x2.columns)



In [ ]:
X_train_df

In [ ]:
#search for a great parameter with GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

max_depth = [3, 6]
criterion_choice = ['squared_error', 'absolute_error']
min_samples_split = [2, 10] 
min_samples_leaf = [2, 5] 

param_grid = {'max_depth': max_depth, 'criterion': criterion_choice, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}

In [ ]:
model = DecisionTreeRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)


In [ ]:
#fit the model
grid_search.fit(X_train, y_train)

In [ ]:
#best r2
print(grid_search.best_score_)

In [ ]:
#best parameters
print(grid_search.best_params_)

In [ ]:
#use the best parameters to fit the model
model = DecisionTreeRegressor(criterion='squared_error', max_depth=6, min_samples_leaf=5, min_samples_split=3)
model.fit(X_train, y_train)


In [ ]:
model.score(X_test, y_test)

In [ ]:
model.score(X_train, y_train)

REJECT LOW MODEL SCORE

USE RANDOM FOREST

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

max_depth_choices = np.random.randint(
    low=1, high=len(X.columns), size=3
)  # A random integer between 1 and the number of columns
criterion_choices = [
    "squared_error",
    "absolute_error",
]  # A list of the possible values optimization metrics
min_samples_split_choices = np.random.randint(
    low=2, high=10, size=3
)  # A random integer between 1 and the number of columns
min_samples_leaf_choices = np.random.randint(
    low=2, high=10, size=3
)  # A random integer between 1 and the number of columns
max_features_choices = np.random.randint(
    low=1, high=len(X.columns), size=3
)  # A random integer between 1 and the number of columns

random_grid = {
    "max_depth": max_depth_choices,
    "criterion": criterion_choices,
    "min_samples_split": min_samples_split_choices,
    "min_samples_leaf": min_samples_leaf_choices,
    "max_features": max_features_choices,
}

In [ ]:
model = DecisionTreeRegressor()
# n_iter is how many random combinations of hyperparameters will test use the computer.
random_search = RandomizedSearchCV(
    estimator=model, param_distributions=random_grid, n_iter=25, cv=5, n_jobs=2
)

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
random_search.best_params_

In [ ]:
random_search.best_score_

In [ ]:
#use random forest for model2

model2 = DecisionTreeRegressor(min_samples_split= 7, min_samples_leaf = 9, max_features = 8, max_depth = 6, criterion = 'squared_error')

model2.fit(X_train, y_train)

model2.score(X_test, y_test)

In [ ]:
model2.score(X_train, y_train)

use model 1

In [ ]:
#fit model
model.fit(X_train, y_train)

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [ ]:
y_train_pred = [round(value, 2) for value in y_train_pred]
y_test_pred = [round(value, 2) for value in y_test_pred]

# Create a dictionary with the results
results = {
    "Set": ["Train"] * len(X_train) + ["Test"] * len(X_test),
    "Real": list(y_train) + list(y_test),
    "Predicted": y_train_pred + y_test_pred,
}

# Create the results DataFrame
results_df = pd.DataFrame(results)
results_df

check most important features that predicts ISI

In [ ]:
importances = model.feature_importances_
feature_importance = pd.DataFrame({"feature": X.columns, "importance": importances})
feature= feature_importance.sort_values(by='importance', ascending=False).reset_index(drop=True)

In [ ]:
display(feature)

In [ ]:
#convert featre importance column to percentage
feature2=feature.copy()
feature2['importance']=feature2['importance']*100

feature2


In [ ]:
# display error margin
results_df["Errors"] = results_df["Real"] - results_df["Predicted"]
results_df.head()

In [ ]:
#select errors where errors is not equal to zero
errors = results_df[results_df["Errors"] != 0]
errors

In [ ]:
#visualize real and predicted values
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10, 5))
sns.scatterplot(data=results_df, x="Real", y="Predicted", hue="Set")
sns.lineplot(data=results_df, x="Real", y="Real", color="black")
plt.title("Real vs Predicted Values")

plt.show()

In [ ]:
# check model performance
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Mean Squared Error
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

# Mean Absolute Error
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# R2 Score
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)


def error_metrics_report(
    y_real_train: list, y_real_test: list, y_pred_train: list, y_pred_test: list
) -> pd.DataFrame:
    """
    The error metrics report function calculates for regression.

     Parameters:
     - y_real_train (list): The actual target values for the training dataset.
     - y_real_test (list): The actual target values for the testing dataset.
     - y_pred_train (list): The predicted target values for the training dataset.
     - y_pred_test (list): The predicted target values for the testing dataset.

     Returns:
     - metrics_df (DataFrame): A Pandas DataFrame containing error metrics for both the training and testing datasets.
     - 'Metric' (str): The name of the error metric.
     - 'Training Set' (float): The error metric value for the training set.
     - 'Testing Set' (float): The error metric value for the testing set.
    """

    MAE_train = mean_absolute_error(y_real_train, y_pred_train)
    MAE_test = mean_absolute_error(y_real_test, y_pred_test)

    # Mean squared error
    MSE_train = mean_squared_error(y_real_train, y_pred_train)
    MSE_test = mean_squared_error(y_real_test, y_pred_test)

    # Root mean squared error
    RMSE_train = mean_squared_error(y_real_train, y_pred_train)
    RMSE_test = mean_squared_error(y_real_test, y_pred_test)

    # R2
    R2_train = r2_score(y_real_train, y_pred_train)
    R2_test = r2_score(y_real_test, y_pred_test)

    results = {
        "Metric": ["MAE", "MSE", "RMSE", "R2"],
        "Train": [MAE_train, MSE_train, RMSE_train, R2_train],
        "Test": [MAE_test, MSE_test, RMSE_test, R2_test],
    }

    results_df = pd.DataFrame(results).round(2)

    pd.set_option("display.float_format", lambda x: "{:.2f}".format(x))

    return results_df

In [ ]:
error_metrics_report(
    list(results_df[results_df["Set"] == "Train"]["Real"]),
    list(results_df[results_df["Set"] == "Test"]["Real"]),
    list(results_df[results_df["Set"] == "Train"]["Predicted"]),
    list(results_df[results_df["Set"] == "Test"]["Predicted"]),
)

The R2 score on the test set was very poor. Subsequent projects will seek to establish a better model. 